In [1]:
%load_ext autoreload

In [2]:
%autoreload
from brainatlas_api.structure_tree import StructureTree
from brainatlas_api.utils import open_json, read_tiff, make_hemispheres_stack
from brainatlas_api.obj_utils import read_obj

from pathlib import Path
import json
from pymesh import obj

In [13]:
class MeshDictionary(dict):
    def __init__(self, mesh_path):
        self.root = Path(mesh_path)
        
        self.files_dict = {int(f.resolve().stem):f for f in self.root.glob("*.obj")}
        
    def __getitem__(self, item):
        if not item in self.keys():
            value = read_obj(self.files_dict[item])
            super().__setitem__(item, value)
            
        return super().__getitem__(item)


class Atlas():
    """ Base class to handle a brainglobe atlas.
        Parameters
        ----------
        path : str or Path object
            path to folder containing data info
        """
    def __init__(self, path):
        self.root = Path(path)
        self.metadata = open_json(self.root / "atlas_metadata.json")

        for attr in ["name", "shape", "resolution"]:
            self.__setattr__(attr, self.metadata[attr])

        structures_list = open_json(self.root / "structures.json")
        self.structures = StructureTree(structures_list)

        self._reference = None
        self._annotated = None
        self._hemispheres = None
        self.acronym_to_id_map = self.structures.get_id_acronym_map()
        
        self.id_to_acronym_map = {v: k for k, v in self.acronym_to_id_map.items()}
        
        self.region_meshes_dict = MeshDictionary(self.root / "meshes")
        

    @property
    def reference(self):
        if self._reference is None:
            self._reference = read_tiff(self.root / "reference.tiff")
        return self._reference

    @property
    def annotated(self):
        if self._annotated is None:
            self._annotated = read_tiff(self.root / "annotated.tiff")
        return self._annotated

    @property
    def hemispheres(self):
        if self._hemispheres is None:
            # If reference is symmetric generate hemispheres block:
            if self.metadata["symmetric"]:
                self._hemispheres = make_hemispheres_stack(self.shape)
            else:
                self._hemispheres = read_tiff(self.root / "hemispheres.tiff")
        return self._hemispheres

    def get_hemisphere_from_coords(self, coords):
        return self.hemispheres[self.idx_from_coords(coords)]

    def get_region_id_from_coords(self, coords):
        return self.annotated[self.idx_from_coords(coords)]
    
    def get_region_name_from_coords(self, coords):
        region_id = self.get_region_id_from_coords(coords)
        
        return self.acronym_map[self.idx_from_coords(coords)]

    def get_mesh_from_id(self, region_id):
        return self.region_meshes_dict[region_id]
    
    def get_mesh_from_name(self, region_name):
        region_id = self.acronym_to_id_map[region_name]
        return self.get_mesh_from_id(region_id)

    def get_brain_mesh(self):
        return self.get_mesh_from_name("root")
    
    @staticmethod
    def idx_from_coords(coords):
        return tuple([int(c) for c in coords])

In [19]:
struct_file_path = Path("/Users/luigipetrucco/.brainglobe/allenbrain_atlas_10um/structures.json")

In [20]:
at = Atlas(struct_file_path.parent)

In [21]:
v = at.get_brain_mesh()

In [22]:
type(v)

tuple

In [23]:
root = struct_file_path.parent / "meshes"

In [43]:
mesh = MeshDictionary(root)

In [45]:
mesh[997]

(array([[ 7896.56 ,  3384.15 ,   503.781],
        [ 7777.44 ,  3414.98 ,   501.766],
        [ 7903.   ,  3480.7  ,   492.858],
        ...,
        [13185.2  ,  4515.53 ,  7052.17 ],
        [13183.2  ,  4503.4  ,  7098.49 ],
        [13184.1  ,  4499.95 ,  7146.95 ]]),
 array([[ 0.00928808, -0.139091  , -0.990236  ],
        [-0.0413177 , -0.123466  , -0.991488  ],
        [ 0.0011034 , -0.0805217 , -0.996752  ],
        ...,
        [-0.951727  ,  0.304431  ,  0.0392041 ],
        [-0.951727  ,  0.304431  ,  0.0392041 ],
        [-0.951727  ,  0.304431  ,  0.0392041 ]]),
 array([[    0,     1,     2],
        [    3,     0,     2],
        [    1,     4,     5],
        ...,
        [49320, 49291, 49319],
        [49291, 49295, 49319],
        [49295, 49290, 49318]]),
 array([[    0,     1,     2],
        [    3,     0,     2],
        [    1,     4,     5],
        ...,
        [49320, 49291, 49319],
        [49291, 49295, 49319],
        [49295, 49290, 49318]]))